## Crear Dataset en el formato necesario

In [1]:
import os
import shutil

from_folder = ["SoccerNet/calibration", "SoccerNet/calibration-2023"]
to_folder = "dataset-detail/images"

# Crear la carpeta destino si no existe
os.makedirs(to_folder, exist_ok=True)

# Iterar sobre cada carpeta de origen
for folder in from_folder:
    dirs = os.listdir(folder)
    folder_tag = os.path.basename(folder)  # Para indicar el origen en el nombre del archivo
    for dir_name in dirs:
        subfolder_path = os.path.join(folder, dir_name)
        to_folder_path = os.path.join(to_folder, dir_name)
        if not os.path.isdir(subfolder_path):
            continue  # Saltar si no es carpeta

        if not os.path.exists(to_folder_path):
            os.makedirs(to_folder_path)

        # Listar todas las imágenes en la subcarpeta
        images = [img for img in os.listdir(subfolder_path) if img.endswith(".jpg") or img.endswith(".png")]

        # Copiar las imágenes a la carpeta destino, agregando el origen al nombre
        for img in images:
            src_path = os.path.join(subfolder_path, img)
            # Nuevo nombre: origen_nombreoriginal
            new_img_name = f"{folder_tag}_{img}"
            dst_path = os.path.join(to_folder_path, new_img_name)
            shutil.copy(src_path, dst_path)

print("Imágenes copiadas correctamente.")

Imágenes copiadas correctamente.


In [2]:
import os
import sys
import json
from_folder = ["SoccerNet/calibration", "SoccerNet/calibration-2023"]
labels_folder = "dataset-detail/labels"

# Diccionario para mapear etiquetas a IDs
class_mapping = {
    "Goal right crossbar": 0,
    "Goal right post left": 1,
    "Goal right post right": 2,
    "Goal left post left": 3,
    "Goal left post left ": 4,
    "Goal left post left": 5,
    "Goal left post right": 6,
    "Goal left crossbar": 7,
    "Small rect. right main": 8,
    "Small rect. right top": 9,
    "Side line right": 10,
    "Side line top": 11,
    "Side line bottom": 12,
    "Big rect. right top": 13,
    "Big rect. right main": 14,
    "Circle right": 15,
    "Side line left": 16,
    "Big rect. left top": 17,
    "Big rect. left bottom": 18,
    "Big rect. left main": 19,
    "Big rect. right bottom": 20,
    "Small rect. right bottom": 21,
    "Small rect. left top": 22,
    "Small rect. left bottom": 23,
    "Small rect. left main": 24,
    "Circle left": 25,
    "Circle central": 26,
    "Middle line": 27,
    "Line unknown": 28,
    "Goal unknown": 29,
}

# Función auxiliar para obtener el bounding box
def get_bbox_from_points(points):
    xs = [max(0, min(1, p["x"])) for p in points]
    ys = [max(0, min(1, p["y"])) for p in points]
    x_min, x_max = min(xs), max(xs)
    y_min, y_max = min(ys), max(ys)
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    width = x_max - x_min
    height = y_max - y_min
    return x_center, y_center, width, height

# Crear la carpeta destino si no existe
os.makedirs(labels_folder, exist_ok=True)

# Iterar sobre cada carpeta de origen
for folder in from_folder:
    dirs = os.listdir(folder)
    folder_tag = os.path.basename(folder)  # Para indicar el origen en el nombre del archivo
    for dir_name in dirs:
        subfolder_path = os.path.join(folder, dir_name)
        to_folder_path = os.path.join(labels_folder, dir_name)
        if not os.path.isdir(subfolder_path):
            continue  # Saltar si no es carpeta

        if not os.path.exists(to_folder_path):
            os.makedirs(to_folder_path)

        # listar todos los json
        jsons = [json_file for json_file in os.listdir(subfolder_path) if json_file.endswith(".json")]

        for j in jsons:
            # si el nombre del archivo no es numérico, saltar
            if not j[:-5].isdigit():
                print(f"Nombre de archivo no numérico: {j}")
                continue
            with open(os.path.join(subfolder_path, j), "r") as f:
                try:
                    data = json.load(f)
                except Exception as e:
                    print(f"Error al cargar el archivo JSON {j}: {e}")
                    continue
                # Nuevo nombre: origen_nombreoriginal.txt
                label_txt = os.path.join(to_folder_path, f"{folder_tag}_{j.replace('.json', '.txt')}")
                with open(label_txt, "w") as label_file:
                    for label, points in data.items():
                        if label not in class_mapping:
                            print(f"Etiqueta desconocida: {label} en {j}")
                        if label not in class_mapping or len(points) < 2:
                            continue
                        class_id = class_mapping[label]
                        x_c, y_c, w, h = get_bbox_from_points(points)
                        label_file.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\n")
print("Etiquetas generadas correctamente.")

Nombre de archivo no numérico: match_info.json
Nombre de archivo no numérico: match_info.json
Nombre de archivo no numérico: match_info.json
Nombre de archivo no numérico: match_info.json
Nombre de archivo no numérico: per_match_info.json
Nombre de archivo no numérico: match_info.json
Nombre de archivo no numérico: per_match_info.json
Nombre de archivo no numérico: match_info.json
Nombre de archivo no numérico: per_match_info.json
Nombre de archivo no numérico: match_info.json
Nombre de archivo no numérico: per_match_info.json
Etiquetas generadas correctamente.


# Finetunear Modelo

In [3]:
from ultralytics import YOLO

# Cargar un modelo base
model = YOLO('yolov8n.pt')  
# Entrenar (fine-tune) con tus datos
model.train(data='line_dataset_detail.yaml', epochs=50, imgsz=640, fliplr=0.0, flipud=0.0)

Ultralytics 8.3.141 🚀 Python-3.12.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060, 7911MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=line_dataset_detail.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train21, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0,

train: Scanning /home/dani/Desktop/futbol/ia/dataset-detail/labels/train.cache... 30976 images, 0 backgrounds, 0 corrupt: 100%|██████████| 30976/30976 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1289.3±280.9 MB/s, size: 159.6 KB)


val: Scanning /home/dani/Desktop/futbol/ia/dataset-detail/labels/valid.cache... 6008 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6008/6008 [00:00<?, ?it/s]


Plotting labels to /home/dani/Desktop/futbol/runs/detect/train21/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /home/dani/Desktop/futbol/runs/detect/train21
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      3.89G     0.5941     0.4395     0.8683        114        640: 100%|██████████| 1936/1936 [03:30<00:00,  9.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 188/188 [00:17<00:00, 10.63it/s]


                   all       6008      54967      0.816      0.686      0.728      0.593

50 epochs completed in 3.332 hours.
Optimizer stripped from /home/dani/Desktop/futbol/runs/detect/train21/weights/last.pt, 6.2MB
Optimizer stripped from /home/dani/Desktop/futbol/runs/detect/train21/weights/best.pt, 6.2MB

Validating /home/dani/Desktop/futbol/runs/detect/train21/weights/best.pt...
Ultralytics 8.3.141 🚀 Python-3.12.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060, 7911MiB)
Model summary (fused): 72 layers, 3,011,498 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 188/188 [00:21<00:00,  8.83it/s]


                   all       6008      54967      0.815      0.687      0.728      0.593
   Goal right crossbar       2249       2249      0.835      0.801      0.821      0.627
  Goal right post left       2216       2216      0.545     0.0722      0.145     0.0529
 Goal right post right       2023       2023      0.551      0.127      0.195      0.071
   Goal left post left       2100       2100      0.763      0.164      0.261      0.101
  Goal left post right       2285       2285      0.676     0.0821      0.165     0.0646
    Goal left crossbar       2379       2379      0.863      0.797      0.841      0.622
Small rect. right main       2570       2570      0.835      0.845      0.877      0.802
 Small rect. right top       2279       2279      0.837      0.789      0.825      0.555
       Side line right       2603       2603      0.863      0.865      0.905      0.851
         Side line top       3611       3611      0.853      0.777       0.83       0.62
      Side line botto

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79f8e7dac7a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0

In [4]:
#guardar el modelo
model.save("yolov8n_line_detailv2.pt")